In [16]:
import os
import sys
sys.path.insert(0, "..")

from tqdm import tqdm

import torch

from my_datasets import *

In [61]:
dataset = AutoregKStepsTokensDataset(
    num_vars = 64,
    num_rules_range = (8, 64),
    ante_prob_range = (0.2, 0.3),
    conseq_prob_range = (0.2, 0.3),
    chain_len_range = (2, 5),
    dataset_len = 1000,
    num_steps = 3
)

In [62]:
item = dataset[0]
tokens = item["tokens"]
labels = item["labels"]
tokens.shape, labels.shape

(torch.Size([64, 129]), torch.Size([3, 64]))

In [63]:
num_dones, cum_label_hots = 0, 0
pbar = tqdm(range(len(dataset)))
for i in pbar:
    item = dataset[i]
    num_dones += 1
    cum_label_hots += item["labels"].sum()
    avg_hots = cum_label_hots / (num_dones * dataset.inner_dataset.num_vars * dataset.num_steps)
    pbar.set_description(f"avghots: {avg_hots:.3f}")

avghots: 0.650: 100%|██████████| 1000/1000 [00:01<00:00, 865.76it/s]
